# Foundations of Computer Science 2023 Project

We first import useful packages

In [35]:
import pandas as pd
import numpy as np
import re
import zipfile

## 1. Extract all trips with busRoute 83

First of all we need to read the dataset

In [2]:
trips=pd.read_csv('metaData.csv')

Let's check what the dataset is like

In [3]:
trips

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.1500
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.1500
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.1500
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.1500
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.808990,0,83,0.739349,279.6404,278.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.785390,0,115,0.434858,279.5283,278.15,292.6655
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.879400,0,102,0.439916,279.1724,277.15,292.1500
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.1500


Let us check what the different routes were

In [4]:
trips['busRoute']

0        -
1       31
2       33
3        -
4       72
        ..
1404    32
1405    83
1406    83
1407    N1
1408    N2
Name: busRoute, Length: 1409, dtype: object

Let's see if it recognizes 83 

In [5]:
trips['busRoute']==83

0       False
1       False
2       False
3       False
4       False
        ...  
1404    False
1405    False
1406    False
1407    False
1408    False
Name: busRoute, Length: 1409, dtype: bool

Since it doesn't (see lines 1405-1406) and there are routes named with letters, the data might be stored in strings

In [6]:
trips['busRoute']=='83'

0       False
1       False
2       False
3       False
4       False
        ...  
1404    False
1405     True
1406     True
1407    False
1408    False
Name: busRoute, Length: 1409, dtype: bool

Ok, that seems to be the case.

In [7]:
trips[trips['busRoute']=='83']

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03T04:42:38Z,1583210558,2020-03-03T19:44:51Z,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06T04:53:23Z,1583470403,2020-03-06T19:44:42Z,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09T14:16:13Z,1583763373,2020-03-09T19:34:17Z,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10T04:50:03Z,1583815803,2020-03-10T19:51:25Z,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12T04:56:41Z,1583989001,2020-03-12T19:44:57Z,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30T04:47:53Z,1669783673,2022-11-30T19:50:22Z,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01T05:19:41Z,1669871981,2022-12-01T18:20:57Z,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02T04:47:48Z,1669956468,2022-12-02T19:40:01Z,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


## 2. Extract all trips where busRoute is not a number

Let us define a function to check if the route is not a number. We have to do this because some routes are just missing (see lines 0 or 3 as example) and some others are letters and numbers (such as 1407-1408)

In [8]:
def check_not_number(route):
    m=re.match('^\d+\d$' ,route)
    if m:
        return False
    else: return True

Let's test it on some strings 

In [9]:
l='lol'
a='83'
b='N1'
c='189492749274398273'
d='18273t87217hh198718198'

In [10]:
check_not_number(l)

True

In [11]:
check_not_number(a)

False

In [12]:
#check_not_number(p)

In [13]:
check_not_number(b)

True

In [14]:
check_not_number(c)

False

In [15]:
check_not_number(d)

True

In [16]:
last_one='12N3'

In [17]:
check_not_number(last_one)

True

Since it seems to work we can use it on the actual dataset

In [18]:
trips[trips['busRoute'].apply(check_not_number)==True]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
9,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,2019-05-10T03:16:11Z,1557458171,2019-05-10T18:51:37Z,1557514297,210577.00,-,1.303391e+09,8.230483,0,43,0.740927,287.5623,282.15,293.15
10,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,2019-05-13T03:10:23Z,1557717023,2019-05-13T23:16:13Z,1557789373,267033.80,-,1.647432e+09,7.891652,0,45,0.804191,284.6764,280.15,293.15
19,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,2019-05-24T02:52:47Z,1558666367,2019-05-24T22:35:11Z,1558737311,263432.60,-,1.448057e+09,7.520249,0,44,0.761068,293.1440,283.15,299.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21T22:38:32Z,1666391912,2022-10-22T02:42:21Z,1666406541,78567.16,N1,4.347766e+08,16.333330,0,45,0.431852,289.2550,288.15,296.15
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22T22:34:45Z,1666478085,2022-10-23T02:29:59Z,1666492199,73427.97,N2,3.997737e+08,17.710530,0,57,0.443358,287.3486,285.15,295.15
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25T23:35:16Z,1669419316,2022-11-26T03:30:39Z,1669433439,72911.26,N2,4.475534e+08,11.216670,1,32,0.465024,281.3884,280.15,293.15
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.15


## 3. For each (busNumber, busRoute) pair, determine the number of trips

Let's check how many different buses number and routes we have:

In [19]:
trips['busNumber'].unique()

array([183, 208])

In [20]:
trips['busRoute'].unique()

array(['-', '31', '33', '72', '46', '32', '83', 'N4', 'N2', 'N1'],
      dtype=object)

Now we can just group by Number and Route and check how many lines we count, dropping the first one since it's referring to a missing busRoute.

In [21]:
trips.groupby(['busNumber', 'busRoute']).size().reset_index(name='Number of trips').drop(0)

,busNumber,busRoute,Number of trips
1,183,31,12
2,183,32,12
3,183,33,130
4,183,46,104
5,183,72,114
6,183,83,441
7,183,N1,10
8,183,N2,19
9,183,N4,11
10,208,31,5


This is in accord with point 2 of this project: the total number of trips for Route 83 is 441 for bus 183 and 405 for bus 208; that is a total of 846 trips, as the lines resulting in from question 1.

## 4. For each trip, compute the ratio between the energy consumption and the average number of passengers 

First of all let's check the data type of energyConsumption: if it is a string we'd have to convert it to a number

In [22]:
type(trips['energyConsumption'][0])

numpy.float64

No work needed here, we can just add a column with the requested information

In [23]:
trips['EnergyRatio']=trips['energyConsumption']/trips['itcs_numberOfPassengers_mean']

In [24]:
trips['EnergyRatio']

0       8.640500e+07
1       1.214747e+07
2       7.342794e+07
3       1.673328e+08
4       2.613190e+07
            ...     
1404    1.070215e+07
1405    5.338462e+07
1406    4.738047e+07
1407    2.247753e+07
1408    3.345769e+07
Name: EnergyRatio, Length: 1409, dtype: float64

## 5. For each station (itcs_stopName), determine the average number of passengers. 

Thanks to the zipfile library, we can access the various excel spreadhseet containing information present in ZTBUS_compressed.zip

In [26]:
namelist=zipfile.ZipFile('https://drive.google.com/drive/u/0/folders/1RL8RyjkngsdMTUdwt_kpu2eQXEW3XojP','r').namelist()
namelist[0]
# B183_2021-04-23_03-47-54_2021-04-23_07-48-48

FileNotFoundError: [Errno 2] No such file or directory: 'https://drive.google.com/drive/u/0/folders/1RL8RyjkngsdMTUdwt_kpu2eQXEW3XojP'

let's try if everything is correct and it opens up the first one:

In [26]:
nome='ZTbus_compressed/'+namelist[0]
a=pd.read_csv(nome)
a

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2019-04-30T03:18:56Z,1556594336,-13.845510,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,251666.7,0.0
1,2019-04-30T03:18:57Z,1556594337,-3.849362,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.3688,254876.2,0.0
2,2019-04-30T03:18:58Z,1556594338,-0.672331,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.9310,251783.3,0.0
3,2019-04-30T03:18:59Z,1556594339,-1.087931,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,255000.0,0.0
4,2019-04-30T03:19:00Z,1556594340,-0.811985,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,253000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19520,2019-04-30T08:44:16Z,1556613856,-0.254938,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19521,2019-04-30T08:44:17Z,1556613857,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19522,2019-04-30T08:44:18Z,1556613858,-0.131183,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0
19523,2019-04-30T08:44:19Z,1556613859,-0.258701,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,0,0,1,286.1500,0.0,0.0


In [27]:
a['itcs_stopName'].unique()

array(['-', 'Zürich, Herdernstrasse', 'Zürich, Hardplatz',
       'Zürich, Güterbahnhof', 'Zürich, Bäckeranlage',
       'Zürich, Militär-/Langstrasse', 'Zürich, Kanonengasse',
       'Zürich, Sihlpost / HB', 'Zürich, Löwenplatz',
       'Zürich, Bahnhofplatz/HB', 'Zürich, Central', 'Zürich, Neumarkt',
       'Zürich, Kunsthaus', 'Zürich, Sprecherstrasse',
       'Zürich, Kreuzplatz', 'Zürich, Signaustrasse',
       'Zürich, Hegibachplatz', 'Zürich, Botanischer Garten',
       'Zürich, Höschgasse', 'Zürich, Fröhlichstrasse',
       'Zürich, Wildbachstrasse', 'Zürich, Bahnhof Tiefenbrunnen',
       'Zürich, Freiestrasse', 'Zürich, Klusplatz',
       'Zürich, Hölderlinsteig', 'Zürich, Klosbach', 'Zürich, Hofstrasse',
       'Zürich, Kirche Fluntern', 'Zürich, Hinterbergstrasse',
       'Zürich, Spyriplatz', 'Zürich, Bethanien', 'Zürich, Toblerplatz'],
      dtype=object)

We notice that we have many different stations name and quite some unnamed stops too; we surely don't need those though.

In [28]:
a=a.drop(a[a['itcs_stopName']=='-'].index)
a

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
246,2019-04-30T03:23:02Z,1556594582,46853.790,423.9126,1.303761,0.827037,0.148511,-,1.0,"Zürich, Herdernstrasse",...,0.917510,0.917510,0.917510,0,1,0,0,291.15,0.0,14819.530
322,2019-04-30T03:24:18Z,1556594658,54698.170,NaN,NaN,NaN,NaN,-,2.0,"Zürich, Hardplatz",...,1.242539,1.242539,1.242539,0,1,0,0,288.15,0.0,16939.120
406,2019-04-30T03:25:42Z,1556594742,33962.440,NaN,NaN,NaN,NaN,-,3.0,"Zürich, Güterbahnhof",...,0.610547,0.610547,0.610547,0,1,0,0,287.15,0.0,6230.346
508,2019-04-30T03:27:24Z,1556594844,59833.090,NaN,NaN,NaN,NaN,-,5.0,"Zürich, Bäckeranlage",...,1.052014,1.052014,1.052014,0,1,0,0,287.15,0.0,20058.230
564,2019-04-30T03:28:20Z,1556594900,11191.580,NaN,NaN,NaN,NaN,-,6.0,"Zürich, Militär-/Langstrasse",...,0.000000,0.000000,0.000000,0,1,0,0,286.15,0.0,12064.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18067,2019-04-30T08:20:03Z,1556612403,-9666.809,NaN,NaN,NaN,NaN,-,0.0,"Zürich, Freiestrasse",...,8.818254,8.887699,8.858202,0,0,0,0,283.15,0.0,-4233.973
18099,2019-04-30T08:20:35Z,1556612435,-98946.970,NaN,NaN,NaN,NaN,-,0.0,"Zürich, Klusplatz",...,4.914641,4.966725,4.937789,0,0,0,0,283.15,0.0,-22060.570
18161,2019-04-30T08:21:37Z,1556612497,21194.270,NaN,NaN,NaN,NaN,-,0.0,"Zürich, Hölderlinsteig",...,0.000000,0.000000,0.000000,0,0,0,0,283.15,0.0,19751.250
18269,2019-04-30T08:23:25Z,1556612605,18611.560,NaN,NaN,NaN,NaN,-,0.0,"Zürich, Klosbach",...,0.000000,0.000000,0.000000,0,0,0,0,283.15,0.0,17867.500


In [29]:
#nome=trips['name']
#for i in range(len(trips)):
#    nome[i]='ZTBUS_compressed/'+nome[i]+'.csv'
#    a=pd.read_csv(nome[i], usecols=['itcs_stopName'])
#    a=a.drop(a[a['itcs_stopName']=='-'].index)
#    stoplist=a['itcs_stopName'].unique()
    # stoplist=a[a['itcs_stopName']!='-']['itcs_stopName'].unique()
#    trips['StopList'][i]=stoplist
#    weights=[]
#    for stop in stoplist:
#        weights.append(str(a[a['itcs_stopName']==stop].count().item()))
#    trips['StopList_weights'][i] = weights

In [30]:
#trips['StopList'][0]==test[test['itcs_stopName']!='-']['itcs_stopName'].unique()

In [31]:
#def check_value (value, listL):
#    for i in range(len(listL)):
#        if value==listL[i]:
#            return i
#    return -1

In [32]:
#value=0
#listL=[1,2,3,4,5,6,0,1,2,4,6,8,6]
#print(check_value (value, listL))
#listL=[1,2,3,4,5,6,1,2,4,6,8,6]
#print(check_value (value, listL))
#listL=['a',2,'ciao',4,5,6,1,2,4,6,8,6]
#print(check_value (value, listL))
#listL=['a',2,'ciao',4,5,6,1,2,4,0,8,6]
#print(check_value (value, listL))
#value='ciao'
#print(check_value (value, listL))

In [33]:
#couples=[]
#stationlist=[]
#for j in range(trips['StopList'][0].size):
#    value=trips['StopList'][0][j]
#    k=int(trips['StopList_weights'][0][j])
#    a=trips['itcs_numberOfPassengers_mean'][0]*k
#    for i in range(len(trips)):
#        listL=trips['StopList'][i]
#        if check_value (value, listL)!=-1:
#            k+=int(trips['StopList_weights'][check_value (value, listL)][j])
#            a+=trips['itcs_numberOfPassengers_mean'][i]*int(trips['StopList_weights'][check_value (value, listL)][j])
#    couple=(value, a/k)
#    if a/k!=0:
#        couples.append(couple)
#    stationlist.append(value)

In [34]:
for i in range(len(namelist)):
    namelist[i]='ZTBus_compressed/'+namelist[i]

In [35]:
namelist

['ZTBus_compressed/B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv',
 'ZTBus_compressed/B183_2019-04-30_13-22-07_2019-04-30_17-54-02.csv',
 'ZTBus_compressed/B183_2019-05-01_05-58-51_2019-05-01_22-32-30.csv',
 'ZTBus_compressed/B183_2019-05-03_02-50-21_2019-05-03_05-53-20.csv',
 'ZTBus_compressed/B183_2019-05-03_15-41-57_2019-05-03_23-06-24.csv',
 'ZTBus_compressed/B183_2019-05-05_07-41-02_2019-05-05_23-20-07.csv',
 'ZTBus_compressed/B183_2019-05-06_03-10-43_2019-05-06_19-20-34.csv',
 'ZTBus_compressed/B183_2019-05-08_02-54-58_2019-05-08_23-00-52.csv',
 'ZTBus_compressed/B183_2019-05-09_03-20-44_2019-05-09_22-52-12.csv',
 'ZTBus_compressed/B183_2019-05-10_03-16-11_2019-05-10_18-51-37.csv',
 'ZTBus_compressed/B183_2019-05-13_03-10-23_2019-05-13_23-16-13.csv',
 'ZTBus_compressed/B183_2019-05-14_03-33-11_2019-05-14_12-48-45.csv',
 'ZTBus_compressed/B183_2019-05-14_13-58-18_2019-05-14_21-04-54.csv',
 'ZTBus_compressed/B183_2019-05-15_03-04-56_2019-05-15_06-58-44.csv',
 'ZTBus_compressed/B

Now we can concatenate each of the 1409 csv files together but dropping the unwanted lines (those with '-' as stopName

In [36]:
previous=pd.read_csv(namelist[0])
previous=previous.drop(previous[previous['itcs_stopName']=='-'].index)
for i in range(len(trips)):
    if i%83==0:
        print("siamo a ", i , "ancora poco: solo ", len(trips)-i, "righe")
    if i+1==len(trips):
        break
    a=pd.read_csv(namelist[i+1])
    a=a.drop(a[a['itcs_stopName']=='-'].index)
    frames=[previous, a]
    previous=pd.concat(frames)

siamo a  0 ancora poco: solo  1409 righe
siamo a  83 ancora poco: solo  1326 righe
siamo a  166 ancora poco: solo  1243 righe
siamo a  249 ancora poco: solo  1160 righe
siamo a  332 ancora poco: solo  1077 righe
siamo a  415 ancora poco: solo  994 righe
siamo a  498 ancora poco: solo  911 righe
siamo a  581 ancora poco: solo  828 righe
siamo a  664 ancora poco: solo  745 righe
siamo a  747 ancora poco: solo  662 righe
siamo a  830 ancora poco: solo  579 righe
siamo a  913 ancora poco: solo  496 righe
siamo a  996 ancora poco: solo  413 righe
siamo a  1079 ancora poco: solo  330 righe
siamo a  1162 ancora poco: solo  247 righe
siamo a  1245 ancora poco: solo  164 righe
siamo a  1328 ancora poco: solo  81 righe


In [37]:
previous

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
246,2019-04-30T03:23:02Z,1556594582,46853.790,423.9126,1.303761,0.827037,0.148511,-,1.0,"Zürich, Herdernstrasse",...,0.917510,0.917510,0.917510,0.0,1.0,0.0,0.0,291.1500,0.0,14819.53000
322,2019-04-30T03:24:18Z,1556594658,54698.170,NaN,NaN,NaN,NaN,-,2.0,"Zürich, Hardplatz",...,1.242539,1.242539,1.242539,0.0,1.0,0.0,0.0,288.1500,0.0,16939.12000
406,2019-04-30T03:25:42Z,1556594742,33962.440,NaN,NaN,NaN,NaN,-,3.0,"Zürich, Güterbahnhof",...,0.610547,0.610547,0.610547,0.0,1.0,0.0,0.0,287.1500,0.0,6230.34600
508,2019-04-30T03:27:24Z,1556594844,59833.090,NaN,NaN,NaN,NaN,-,5.0,"Zürich, Bäckeranlage",...,1.052014,1.052014,1.052014,0.0,1.0,0.0,0.0,287.1500,0.0,20058.23000
564,2019-04-30T03:28:20Z,1556594900,11191.580,NaN,NaN,NaN,NaN,-,6.0,"Zürich, Militär-/Langstrasse",...,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,286.1500,0.0,12064.05000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12542,2022-12-11T03:03:48Z,1670727828,12677.750,464.3928,0.000000,0.827626,0.148469,N2,9.0,"Zürich, Zehntenhausplatz",...,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,274.1500,100040.5,10472.81000
12591,2022-12-11T03:04:37Z,1670727877,5376.588,458.3075,0.000000,0.827651,0.148482,N2,0.0,"Zürich, Bahnhof Affoltern",...,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,274.1500,245000.0,489.56790
12636,2022-12-11T03:05:22Z,1670727922,-55071.570,456.9075,2.076901,0.827671,0.148520,N2,0.0,"Zürich, Fronwald",...,8.008564,8.035274,8.016577,0.0,0.0,0.0,0.0,274.1500,0.0,-9181.79800
12674,2022-12-11T03:06:00Z,1670727960,-22885.870,452.2150,0.357920,0.827700,0.148547,N2,0.0,"Zürich, Unteraffoltern",...,8.735694,8.754341,8.727656,0.0,0.0,0.0,0.0,274.1500,0.0,-5328.51800


Now all that is left is to group by stop name and calculate the average

In [38]:
previous.groupby('itcs_stopName').mean('itcs_numberOfPassengers')['itcs_numberOfPassengers']

itcs_stopName
Zürich, Albisrank                 16.196598
Zürich, Albisriederplatz          25.453398
Zürich, Altes Krematorium         21.980348
Zürich, Bahnhof Affoltern          2.850746
Zürich, Bahnhof Altstetten         7.816580
                                    ...    
Zürich, Zentrum Witikon           17.255102
Zürich, Zweiackerstrasse           7.502203
Zürich, Zwielplatz                 2.049383
Zürich, Zwinglihaus               23.196668
Zürich,Kalkbreite/Bhf.Wiedikon    23.160748
Name: itcs_numberOfPassengers, Length: 149, dtype: float64

We can check that it is correct by calculating the number of unique stop names 

In [39]:
len(previous['itcs_stopName'].unique())

149

## 6. For each station, determine the buses that have stopped there at least once.

In [40]:
previous.columns

Index(['time_iso', 'time_unix', 'electric_powerDemand', 'gnss_altitude',
       'gnss_course', 'gnss_latitude', 'gnss_longitude', 'itcs_busRoute',
       'itcs_numberOfPassengers', 'itcs_stopName',
       'odometry_articulationAngle', 'odometry_steeringAngle',
       'odometry_vehicleSpeed', 'odometry_wheelSpeed_fl',
       'odometry_wheelSpeed_fr', 'odometry_wheelSpeed_ml',
       'odometry_wheelSpeed_mr', 'odometry_wheelSpeed_rl',
       'odometry_wheelSpeed_rr', 'status_doorIsOpen', 'status_gridIsAvailable',
       'status_haltBrakeIsActive', 'status_parkBrakeIsActive',
       'temperature_ambient', 'traction_brakePressure',
       'traction_tractionForce'],
      dtype='object')

In [41]:
previous

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
246,2019-04-30T03:23:02Z,1556594582,46853.790,423.9126,1.303761,0.827037,0.148511,-,1.0,"Zürich, Herdernstrasse",...,0.917510,0.917510,0.917510,0.0,1.0,0.0,0.0,291.1500,0.0,14819.53000
322,2019-04-30T03:24:18Z,1556594658,54698.170,NaN,NaN,NaN,NaN,-,2.0,"Zürich, Hardplatz",...,1.242539,1.242539,1.242539,0.0,1.0,0.0,0.0,288.1500,0.0,16939.12000
406,2019-04-30T03:25:42Z,1556594742,33962.440,NaN,NaN,NaN,NaN,-,3.0,"Zürich, Güterbahnhof",...,0.610547,0.610547,0.610547,0.0,1.0,0.0,0.0,287.1500,0.0,6230.34600
508,2019-04-30T03:27:24Z,1556594844,59833.090,NaN,NaN,NaN,NaN,-,5.0,"Zürich, Bäckeranlage",...,1.052014,1.052014,1.052014,0.0,1.0,0.0,0.0,287.1500,0.0,20058.23000
564,2019-04-30T03:28:20Z,1556594900,11191.580,NaN,NaN,NaN,NaN,-,6.0,"Zürich, Militär-/Langstrasse",...,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,286.1500,0.0,12064.05000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12542,2022-12-11T03:03:48Z,1670727828,12677.750,464.3928,0.000000,0.827626,0.148469,N2,9.0,"Zürich, Zehntenhausplatz",...,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,274.1500,100040.5,10472.81000
12591,2022-12-11T03:04:37Z,1670727877,5376.588,458.3075,0.000000,0.827651,0.148482,N2,0.0,"Zürich, Bahnhof Affoltern",...,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,274.1500,245000.0,489.56790
12636,2022-12-11T03:05:22Z,1670727922,-55071.570,456.9075,2.076901,0.827671,0.148520,N2,0.0,"Zürich, Fronwald",...,8.008564,8.035274,8.016577,0.0,0.0,0.0,0.0,274.1500,0.0,-9181.79800
12674,2022-12-11T03:06:00Z,1670727960,-22885.870,452.2150,0.357920,0.827700,0.148547,N2,0.0,"Zürich, Unteraffoltern",...,8.735694,8.754341,8.727656,0.0,0.0,0.0,0.0,274.1500,0.0,-5328.51800


In [42]:
trips.head(5)

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,EnergyRatio
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15,8.640500e+07
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.15,1.214747e+07
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.15,7.342794e+07
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15,1.673328e+08
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.15,2.613190e+07


in previous 
-> aggiungo colonna con numerobus 
-> raggruppi per stopname e in caso per numerobus 
-> conti se in ogni gruppetto c'è solo 183 oopure 208 oppure entrambi 

In [43]:
previous.head()

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
246,2019-04-30T03:23:02Z,1556594582,46853.79,423.9126,1.303761,0.827037,0.148511,-,1.0,"Zürich, Herdernstrasse",...,0.917510,0.917510,0.917510,0.0,1.0,0.0,0.0,291.15,0.0,14819.530
322,2019-04-30T03:24:18Z,1556594658,54698.17,NaN,NaN,NaN,NaN,-,2.0,"Zürich, Hardplatz",...,1.242539,1.242539,1.242539,0.0,1.0,0.0,0.0,288.15,0.0,16939.120
406,2019-04-30T03:25:42Z,1556594742,33962.44,NaN,NaN,NaN,NaN,-,3.0,"Zürich, Güterbahnhof",...,0.610547,0.610547,0.610547,0.0,1.0,0.0,0.0,287.15,0.0,6230.346
508,2019-04-30T03:27:24Z,1556594844,59833.09,NaN,NaN,NaN,NaN,-,5.0,"Zürich, Bäckeranlage",...,1.052014,1.052014,1.052014,0.0,1.0,0.0,0.0,287.15,0.0,20058.230
564,2019-04-30T03:28:20Z,1556594900,11191.58,NaN,NaN,NaN,NaN,-,6.0,"Zürich, Militär-/Langstrasse",...,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,286.15,0.0,12064.050


In [45]:
prova = pd.read_csv('ZTBus_compressed/B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv')
prova.head(3)

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_mr,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce
0,2019-04-30T03:18:56Z,1556594336,-13.845510,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,293.1500,251666.7,0.0
1,2019-04-30T03:18:57Z,1556594337,-3.849362,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.3688,254876.2,0.0
2,2019-04-30T03:18:58Z,1556594338,-0.672331,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,0.0,1,1,0,0,292.9310,251783.3,0.0


In [46]:
nomi = trips['name']
nomi

0       B183_2019-04-30_03-18-56_2019-04-30_08-44-20
1       B183_2019-04-30_13-22-07_2019-04-30_17-54-02
2       B183_2019-05-01_05-58-51_2019-05-01_22-32-30
3       B183_2019-05-03_02-50-21_2019-05-03_05-53-20
4       B183_2019-05-03_15-41-57_2019-05-03_23-06-24
                            ...                     
1404    B208_2022-12-06_14-43-49_2022-12-06_18-22-52
1405    B208_2022-12-07_05-13-02_2022-12-07_19-19-53
1406    B208_2022-12-08_05-22-20_2022-12-08_18-39-15
1407    B208_2022-12-09_23-55-12_2022-12-10_03-24-28
1408    B208_2022-12-10_23-34-46_2022-12-11_03-29-05
Name: name, Length: 1409, dtype: object

In [47]:
nome

'ZTbus_compressed/B183_2019-04-30_03-18-56_2019-04-30_08-44-20.csv'

In [48]:
prova['numerbus'] = 0

In [49]:
prova.head(2)

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce,numerbus
0,2019-04-30T03:18:56Z,1556594336,-13.845510,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,1,1,0,0,293.1500,251666.7,0.0,0
1,2019-04-30T03:18:57Z,1556594337,-3.849362,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,1,1,0,0,292.3688,254876.2,0.0,0


In [ ]:
m = re.match('^ZTBUS/B183',nome[0])
if m:
    prova['numerobus'] = 183
else:
    prova['numerbus'] = 208

In [ ]:
prova.head(2)

In [ ]:
previous['nbus'] = 0
previous.head(2)

In [ ]:
m = re.match('^ZTBUS/B183',nome[0])
if m:
    previous['nbus'] = 183
else:
    previous['nbus'] = 208

In [ ]:
previous.head(3)

In [ ]:
for i in range(len(trips)):
    m = re.match('ZTBUS/B183',nome[i])
    if m != 'None':
        previous['nbus'] = 183
    else:
        previous['nbus'] = 208

In [ ]:
previous

## 9. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance.

We can just group by route and bus number and then sum over energy consumption and driven distance:

In [50]:
grouped = trips.groupby(['busRoute','busNumber']).sum()[['energyConsumption', 'drivenDistance']]
grouped

energyConsumption  drivenDistance
busRoute busNumber                                   
-        183             1.245498e+10      1999119.75
31       183             1.387782e+10      2279952.00
         208             5.535865e+09      1046306.68
32       183             1.393320e+10      2256540.10
         208             1.283340e+10      2337064.21
33       183             1.761996e+11     29510481.41
         208             3.474842e+10      6161892.39
46       183             1.026237e+11     18224657.94
         208             2.337366e+10      4186493.70
72       183             1.394368e+11     23637238.45
         208             5.100382e+10      9427138.15
83       183             2.995540e+11     51249813.70
         208             2.907837e+11     49965928.79
N1       183             4.217283e+09       705122.71
         208             2.316328e+09       410691.93
N2       183             7.929098e+09      1390618.35
         208             7.913191e+09      1464008.67
N4       183             4.720068e+09       766868.89
         208             3.152512e+09       519599.97

Now let's just create a new column with the wanted ratio:

In [51]:
grouped['ratio'] = grouped['energyConsumption'] / grouped['drivenDistance']
grouped

energyConsumption  drivenDistance        ratio
busRoute busNumber                                                
-        183             1.245498e+10      1999119.75  6230.233181
31       183             1.387782e+10      2279952.00  6086.891785
         208             5.535865e+09      1046306.68  5290.862713
32       183             1.393320e+10      2256540.10  6174.585331
         208             1.283340e+10      2337064.21  5491.247671
33       183             1.761996e+11     29510481.41  5970.747507
         208             3.474842e+10      6161892.39  5639.244537
46       183             1.026237e+11     18224657.94  5631.035959
         208             2.337366e+10      4186493.70  5583.110707
72       183             1.394368e+11     23637238.45  5899.029643
         208             5.100382e+10      9427138.15  5410.318878
83       183             2.995540e+11     51249813.70  5844.977154
         208             2.907837e+11     49965928.79  5819.638891
N1       183             4.217283e+09       705122.71  5980.920966
         208             2.316328e+09       410691.93  5640.061883
N2       183             7.929098e+09      1390618.35  5701.850332
         208             7.913191e+09      1464008.67  5405.152963
N4       183             4.720068e+09       766868.89  6154.986937
         208             3.152512e+09       519599.97  6067.190112

## 10. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe

First we extract maximum value of our ratio for each route:

In [52]:
grouped_max= grouped.loc[grouped['ratio'].groupby('busRoute').idxmax()]
grouped_max

,,energyConsumption,drivenDistance,ratio
busRoute,busNumber,,,
-,183,1.245498e+10,1999119.75,6230.233181
31,183,1.387782e+10,2279952.00,6086.891785
32,183,1.393320e+10,2256540.10,6174.585331
33,183,1.761996e+11,29510481.41,5970.747507
46,183,1.026237e+11,18224657.94,5631.035959
72,183,1.394368e+11,23637238.45,5899.029643
83,183,2.995540e+11,51249813.70,5844.977154
N1,183,4.217283e+09,705122.71,5980.920966
N2,183,7.929098e+09,1390618.35,5701.850332


Then we do the same for the minimum ratio value over the routes:

In [53]:
grouped_min=grouped.loc[grouped['ratio'].groupby('busRoute').idxmin()]
grouped_min

,,energyConsumption,drivenDistance,ratio
busRoute,busNumber,,,
-,183,1.245498e+10,1999119.75,6230.233181
31,208,5.535865e+09,1046306.68,5290.862713
32,208,1.283340e+10,2337064.21,5491.247671
33,208,3.474842e+10,6161892.39,5639.244537
46,208,2.337366e+10,4186493.70,5583.110707
72,208,5.100382e+10,9427138.15,5410.318878
83,208,2.907837e+11,49965928.79,5819.638891
N1,208,2.316328e+09,410691.93,5640.061883
N2,208,7.913191e+09,1464008.67,5405.152963


We then merge the two:

In [54]:
grouped_tot = pd.merge(grouped_max, grouped_min, on='busRoute')
grouped_tot

,energyConsumption_x,drivenDistance_x,ratio_x,energyConsumption_y,drivenDistance_y,ratio_y
busRoute,,,,,,
-,1.245498e+10,1999119.75,6230.233181,1.245498e+10,1999119.75,6230.233181
31,1.387782e+10,2279952.00,6086.891785,5.535865e+09,1046306.68,5290.862713
32,1.393320e+10,2256540.10,6174.585331,1.283340e+10,2337064.21,5491.247671
33,1.761996e+11,29510481.41,5970.747507,3.474842e+10,6161892.39,5639.244537
46,1.026237e+11,18224657.94,5631.035959,2.337366e+10,4186493.70,5583.110707
72,1.394368e+11,23637238.45,5899.029643,5.100382e+10,9427138.15,5410.318878
83,2.995540e+11,51249813.70,5844.977154,2.907837e+11,49965928.79,5819.638891
N1,4.217283e+09,705122.71,5980.920966,2.316328e+09,410691.93,5640.061883
N2,7.929098e+09,1390618.35,5701.850332,7.913191e+09,1464008.67,5405.152963


And now we have the difference simply by:

In [55]:
grouped_tot['ratio_x'] - grouped_tot['ratio_y']

busRoute
-       0.000000
31    796.029072
32    683.337660
33    331.502969
46     47.925252
72    488.710765
83     25.338263
N1    340.859083
N2    296.697369
N4     87.796825
dtype: float64

## 14. For each month of the year, compute the average ambient temperature

First of all we transform the temperature to celsius, for easier understanding:

In [56]:
for i in range(len(trips)):
    trips['temperature_ambient_mean'][i]=trips['temperature_ambient_mean'][i]-273.15

/var/folders/y0/c69z4ct1695f_69pbyjfvw3h0000gn/T/ipykernel_41005/3124311089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips['temperature_ambient_mean'][i]=trips['temperature_ambient_mean'][i]-273.15
/var/folders/y0/c69z4ct1695f_69pbyjfvw3h0000gn/T/ipykernel_41005/3124311089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips['temperature_ambient_mean'][i]=trips['temperature_ambient_mean'][i]-273.15
/var/folders/y0/c69z4ct1695f_69pbyjfvw3h0000gn/T/ipykernel_41005/3124311089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,EnergyRatio
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30T03:18:56Z,1556594336,2019-04-30T08:44:20Z,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,9.2280,281.15,293.1500,8.640500e+07
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30T13:22:07Z,1556630527,2019-04-30T17:54:02Z,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,14.3943,285.15,293.1500,1.214747e+07
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01T05:58:51Z,1556690331,2019-05-01T22:32:30Z,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,15.5990,280.15,294.1500,7.342794e+07
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03T02:50:21Z,1556851821,2019-05-03T05:53:20Z,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,9.2629,281.15,292.1500,1.673328e+08
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03T15:41:57Z,1556898117,2019-05-03T23:06:24Z,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,11.5825,282.15,287.1500,2.613190e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06T14:43:49Z,1670337829,2022-12-06T18:22:52Z,1670350972,51798.78,32,4.260419e+08,39.808990,0,83,0.739349,6.4904,278.15,291.1500,1.070215e+07
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07T05:13:02Z,1670389982,2022-12-07T19:19:53Z,1670440793,210041.60,83,1.536697e+09,28.785390,0,115,0.434858,6.3783,278.15,292.6655,5.338462e+07
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08T05:22:20Z,1670476940,2022-12-08T18:39:15Z,1670524755,190372.70,83,1.415700e+09,29.879400,0,102,0.439916,6.0224,277.15,292.1500,4.738047e+07
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09T23:55:12Z,1670630112,2022-12-10T03:24:28Z,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,6.3040,277.15,291.1500,2.247753e+07


Now let's set the date:

In [58]:
trips['startTime_iso']=pd.to_datetime(trips['startTime_iso'])

Let's put the month in a new columm:

In [59]:
trips['mese']=trips['startTime_iso'].dt.month

And we have the result:

In [60]:
trips.groupby('mese')['temperature_ambient_mean'].mean().reset_index(name='avg_temp')

,mese,avg_temp
0,1,5.808474
1,2,7.379928
2,3,10.169157
3,4,13.458216
4,5,16.901582
5,6,22.993297
6,7,24.226967
7,8,22.697943
8,9,19.212038
9,10,14.564072
